# 2.2.1 Planzeichnung – OCR (GPT4o)

* Zero-Shot
* Chain-Of-Thought

In [10]:
from utils.parser import Parser
from utils.openai import OpenAI
from utils.pprint import pprint
import asyncio

instructions = "Du bist ein Assistent um Textinformationen aus einer Planzeichnung eines Bebauungsplans zu extrahieren. Achte auf Vollständigkeit."
jarvis = OpenAI(instructions)
parser = Parser()

In [13]:
%store -r msg211_l04_zebb_nz
jarvis.clearContext()
jarvis.addContext([
    # Output_1_PZE (best-case-scenario)
    {
        "type": "text",
        "text": f"Tabellenerklärung/Schema der Nutzungsschablone: {msg211_l04_zebb_nz}"
    },
    # Output_2_PZE (best-case-scenario)
    # {
    #     "type": "text",
    #     "text": 'Erklärung der Begriffe der Nutzungsschablone im JSON-Format: {"Art der Nutzung": {"Definition": "Gibt die Art der Nutzung des Grundstücks an, z.B. Gewerbegebiet.","Beispiel": "GE (Gewerbegebiet)"},"Gebäudehöhe": {"Definition": "Maximal zulässige Gebäudehöhe über Erdgeschossfußbodenhöhe.","Beispiel": "GH=max.12,00"},"Grundflächenzahl": {"Definition": "Gibt das Verhältnis der bebauten Fläche zur Grundstücksfläche an.","Beispiel": "0,6"},"Geschossflächenzahl": {"Definition": "Gibt das Verhältnis der gesamten Geschossfläche zur Grundstücksfläche an.","Beispiel": "1,2"},"Bauweise": {"Definition": "Gibt die Art und Weise der Bebauung an, z.B. abweichende Bauweise.","Beispiel": "a (abweichende Bauweise)"}}'
    # },
])

In [14]:
# Planzeichnung: Nutzungsschablone – WITHOUT BOUNDING BOX
pz_crop_path = "../data/processed/bpläne/1_alles_in_einem_dokument/L04-01-PZ-crop.png"

instruction = 'Gehe Schritt für Schritt vor: 1. Finde die Nutzungsschablonen in der vorliegenden Planzeichnung. 2. Gebe die Nutzungsschablone jeweils im JSON-Format aus.'

messages = await jarvis.extractTextFromImages(instruction, [pz_crop_path])
pprint(messages)

1. **Finde die Nutzungsschablonen in der vorliegenden Planzeichnung:**

Die Nutzungsschablonen sind in der Planzeichnung durch rot umrandete Tabellen dargestellt. Diese Tabellen enthalten Informationen zu Baugebiet, Zahl der Vollgeschosse, Grundflächenzahl, Geschossflächenzahl, Bauweise, Dachform und Dachneigung.

2. **Gebe die Nutzungsschablone jeweils im JSON-Format aus:**

Hier sind die Nutzungsschablonen, die in der Planzeichnung gefunden wurden, im JSON-Format:

```json
{
  "Nutzungsschablonen": [
    {
      "Baugebiet": "WA",
      "Zahl der Vollgeschosse": "II",
      "Grundflächenzahl": "0.4",
      "Geschossflächenzahl": "0.8",
      "Bauweise": "offen",
      "Dachform und Dachneigung": "Satteldach, 30-45°"
    },
    {
      "Baugebiet": "MI",
      "Zahl der Vollgeschosse": "III",
      "Grundflächenzahl": "0.6",
      "Geschossflächenzahl": "1.2",
      "Bauweise": "geschlossen",
      "Dachform und Dachneigung": "Flachdach"
    },
    {
      "Baugebiet": "GE",
      "Za

In [15]:
# Planzeichnung: Nutzungsschablone – WITH BOUNDING BOX
pz_crop_path = "../data/processed/bpläne/1_alles_in_einem_dokument/L04-01-PZ-bb-crop.png"

instruction = 'Gehe Schritt für Schritt vor: 1. Finde alle rot umrandeten Nutzungsschablonen in der vorliegenden Planzeichnung. 2. Gebe die Nutzungsschablonen jeweils im JSON-Format aus: {<Parameter>:<Wert>}'

messages = await jarvis.extractTextFromImages(instruction, [pz_crop_path])
pprint(messages)

Hier sind die rot umrandeten Nutzungsschablonen aus der Planzeichnung im JSON-Format:

```json
[
    {
        "Baugebiet": "WA",
        "Zahl der Vollgeschosse": "II",
        "Grundflächenzahl": "0.4",
        "Geschossflächenzahl": "0.8",
        "Bauweise": "offen",
        "Dachform und Dachneigung": "Satteldach 30-45°"
    },
    {
        "Baugebiet": "WA",
        "Zahl der Vollgeschosse": "II",
        "Grundflächenzahl": "0.4",
        "Geschossflächenzahl": "0.8",
        "Bauweise": "offen",
        "Dachform und Dachneigung": "Satteldach 30-45°"
    },
    {
        "Baugebiet": "WA",
        "Zahl der Vollgeschosse": "II",
        "Grundflächenzahl": "0.4",
        "Geschossflächenzahl": "0.8",
        "Bauweise": "offen",
        "Dachform und Dachneigung": "Satteldach 30-45°"
    },
    {
        "Baugebiet": "WA",
        "Zahl der Vollgeschosse": "II",
        "Grundflächenzahl": "0.4",
        "Geschossflächenzahl": "0.8",
        "Bauweise": "offen",
        "Dachf

In [17]:
# Planzeichnung: Nutzungsschablone – VISUAL SEARCH + CROPPING
base64_ze, img_size = parser.imageToBase64("../data/processed/bpläne/1_alles_in_einem_dokument/L04-01-ZE-bb-crop.png")
image_url_prompts = []
for index in range(20):
    base64_nz, img_size = parser.imageToBase64(f'../data/processed/bpläne/1_alles_in_einem_dokument/L04-01-PZ-nz{index+1}-crop.png')
    image_url_prompts.append({
        "type": "image_url",
        "image_url": {
            "url": f"data:image/jpeg;base64,{base64_nz}",
            "detail": "high"
        }
    })

# jarvis.addContext([
#     # Output (best-case-scenario)
#     {
#         "type": "text",
#         "text": "Zur Lösung jeder Aufgabe ist die bereitgestellte Zeichenerklärung zu verwenden, um alle vorkommenden Zeichen korrekt erkennen bzw. interpretieren zu können."
#     },
# ])

message = jarvis.request([
    # {
    #     "type": "text",
    #     "text": 'Hier ist die Zeichenerklärung:'
    # },
    # {
    #     "type": "image_url",
    #     "image_url": {
    #         "url": f"data:image/jpeg;base64,{base64_ze}",
    #         "detail": "high"
    #     }
    # },
    {
        "type": "text",
        "text": 'Hier sind die vorkommenden Nutzungsschablonen der Planzeichnung:'
    },
    *image_url_prompts,
    {
        "type": "text",
        "text": 'Gebe die Werte der jeweiligen Nutzungsschablonen strukturiert im JSON-Format aus: {<Parameter>:<Wert>}'
    }
])

print(message)
# jarvis.clearContext()

Hier sind die Werte der jeweiligen Nutzungsschablonen im JSON-Format:

```json
[
  {
    "Baugebiet": "GE 2",
    "Zahl der Vollgeschosse": "II",
    "Grundflächenzahl": "0,6",
    "Geschossflächenzahl": "1,2",
    "Bauweise": "0",
    "Dachform und Dachneigung": "SD DN 12-18"
  },
  {
    "Baugebiet": "WA",
    "Zahl der Vollgeschosse": "I",
    "Grundflächenzahl": "0,3",
    "Geschossflächenzahl": "0,5",
    "Bauweise": "0",
    "Dachform und Dachneigung": "SD DN 28-33"
  },
  {
    "Baugebiet": "WA",
    "Zahl der Vollgeschosse": "I",
    "Grundflächenzahl": "0,3",
    "Geschossflächenzahl": "0,5",
    "Bauweise": "0",
    "Dachform und Dachneigung": "SD DN 28-33"
  },
  {
    "Baugebiet": "WA",
    "Zahl der Vollgeschosse": "I",
    "Grundflächenzahl": "0,3",
    "Geschossflächenzahl": "0,5",
    "Bauweise": "0",
    "Dachform und Dachneigung": "SD DN 28-33"
  },
  {
    "Baugebiet": "MI",
    "Zahl der Vollgeschosse": "II",
    "Grundflächenzahl": "0,4",
    "Geschossflächenzahl":